In [1]:
from DataCollector import DataCollectorv2
from Dataset import DatasetHPs
from NNModel import NNModelHPs
from covit import CovitProject

In [2]:
dc = DataCollectorv2()

Building Data frame
Done building Data frame
Building remote dicts
Done building remote dicts
Building local dicts
Done building local dicts


In [ ]:
dc.downloadLineages(2048,2048)

java -jar cdp-file-downloader.jar --domain=VIRAL_SEQUENCES --datatype=SEQUENCES --format=FASTA --location=../data/tmp/ --email=NONE --accessions=OA971657,OU200296,OD913372,MW844875,OA966097,OA969432,OA967192,OU128788,OA980250,LR861666,MT806792,OU088328,OU082079,MZ907247,MW365042,LR824476,OD916910,LC572068,OM251163,FR999516,OA963930,FR996522,OU008751,LR814214,FR993979,OU806760,LR880579,MZ071875,OA969962,OM462099,OU171293,OK372382,MZ001835,MT609918,MW933514,OE999708,OA976870,FR993702,OU162146,FR998077,FR998876,OC996760,LR963197,MW778956,OA967619,FR995559,OA976030,OU173577,OU369955,FR994328,OU277671,OA978175,OA979024,OA969532,OA993413,LR963233,OA980932,OU391388,OU800092,OU448558,OU368781,MT479226,FR999865,MZ230715,OA972358,OE995665,FR994838,MW902967,OV079428,OU073634,OA969242,MZ072793,FR994429,OA979055,MZ341177,OA993741,OE998906,MZ905706,OA963912,MW781252,OU093067,OA964620,OK232869,MZ332412,MT646108,OA963989,MW134564,MZ143117,OU082258,FR996376,LR963206,OM725541,OU088891,OU071858,OK423946,

Download completed to ../data/tmp//viral_sequences/sequences/fasta/20220710-1630.fasta.
java -jar cdp-file-downloader.jar --domain=VIRAL_SEQUENCES --datatype=SEQUENCES --format=FASTA --location=../data/tmp/ --email=NONE --accessions=FR999104,OA973749,FR990360,MW871527,OA981365,MT460138,MT396243,FR994587,OA969522,MZ001434,OA966719,OU200304,FR995357,OA977917,MW751137,OB999610,OK435564,FR993642,OA981608,OA965863,MT973480,LR821911,OK424017,OA973548,OU809874,FR994012,MW984941,FR994658,FR994234,MT834310,MT831893,OD921145,OA971484,OE998766,OA970456,LR821834,LR883091,MW674847,OA976301,MW040684,FR995506,MW868725,MZ073163,OA979608,MW641216,OA981167,OU326239,OU806026,OM759866,OA977243,MW134004,OU793763,FR994322,OA980263,OA967625,OA979026,OB986848,OU057451,MZ284568,FR997993,OA963983,OU280663,OA975805,MZ171792,FR994285,OA968680,FR998276,OU041425,MT821666,OA975988,OK435768,MW340911,OA975652,OU806370,OA973509,MW134229,OE995985,MW674854,LR898702,OA971416,OA966386,OV081720,OA966821,OA973079,OA967371,MW

Download completed to ../data/tmp//viral_sequences/sequences/fasta/20220710-1630.fasta.
java -jar cdp-file-downloader.jar --domain=VIRAL_SEQUENCES --datatype=SEQUENCES --format=FASTA --location=../data/tmp/ --email=NONE --accessions=ON339725,FR994370,FR998048,MW976890,OL588209,OA991595,OL790191,MW752951,OU083625,OM986916,OA976222,OM260998,OA981084,OA981683,FR995464,OU083119,OK657773,OU093051,OA964932,OU222920,OA976150,MZ036139,MW907482,LR878222,MW181750,FR995273,LR881587,OU200300,FR999144,FR995377,OU000517,OA970286,OA976090,OA968956,MZ918818,ON051710,FR997124,FR998805,OK425338,OA984577,OA980608,OA974457,MW673073,MZ167029,OU277966,OA990753,MW273862,LR883656,OA967316,FR993845,MZ895755,LR882860,FR999056,OD913801,OU084391,MZ473220,MW870739,MW191079,OA990320,LR860591,FR999240,MW974507,OA976845,FR997291,OU117874,MW836869,MZ332391,OU814331,MT679199,MT886396,MW598435,MZ230689,MW859152,OU007296,OA967529,OA977042,OL615289,OU081285,OU796043,MT890212,MZ150539,MW643423,OM485363,OA964303,OU174689,MZ

Download completed to ../data/tmp//viral_sequences/sequences/fasta/20220710-1632.fasta.
java -jar cdp-file-downloader.jar --domain=VIRAL_SEQUENCES --datatype=SEQUENCES --format=FASTA --location=../data/tmp/ --email=NONE --accessions=LR991954,OA971268,OU167075,OU163174,MW781167,OD986800,OA964698,OA976289,MW190808,OA979692,OA982761,FR998604,OA993462,OA968605,MW870709,FR995267,OA993815,FR997975,FR994751,OU813076,FR993974,MZ916026,OU062290,OL588201,OA964691,MW564884,OA970079,OB986835,OK426841,OA981089,MW309076,OU805107,LR880586,OL825777,OK436444,OA980556,MW945075,OU797210,OU490198,MW908700,OA984333,OD901652,OA970314,OA966051,OU181102,OK510442,OA980129,MZ916585,OM720907,OA963975,OA980378,OA978764,LR878212,OU100787,OA979775,MT460140,OU191314,OU090032,MZ332227,LC556316,OA975736,MW191309,OU173335,OK465989,MZ332066,OU813527,OD899629,FR994788,FR997032,LR824277,MW739085,LR821993,LC636480,FR990762,OV087991,OU164333,OA964732,LR821842,OA969935,MZ635905,OU806338,OE999424,FR995589,MW751290,MZ332418,OU

Download completed to ../data/tmp//viral_sequences/sequences/fasta/20220710-1632.fasta.
java -jar cdp-file-downloader.jar --domain=VIRAL_SEQUENCES --datatype=SEQUENCES --format=FASTA --location=../data/tmp/ --email=NONE --accessions=MT951973,MT459891,MZ332480,MZ332083,OA966481,OA965166,OU075694,MW637555,OA971910,FR999127,MZ332203,MT820470,OK372397,LC623947,MT418894,OU175270,OA970493,OA964601,MW863949,OA980280,OU109418,OA980432,OA978492,OA981276,MZ895725,BS000890,MT834624,OA967721,OK435476,MZ918398,OU080955,MW030219,LR877773,OU161852,OU026751,MT972123,OA964661,FR999237,MT890326,OA971083,OK487821,MZ906587,OB989873,OA967791,OA973307,MW779084,LR821924,OA980977,OA969987,OL308722,MZ905682,OU181400,OA981126,MZ332368,LR878233,MW211003,OA979783,MW521666,MT882321,OA980007,LR884413,GWHAOOK01000001,OA968266,MW191190,MW093607,OA980640,MZ518153,FR995205,OA964628,OU159969,OU080243,MT371568,OA976308,OV087910,MW738883,OA966079,OA970484,OA980057,FR994357,OU093478,OD925827,OA965126,OU389294,MW309440,ON16

Download completed to ../data/tmp//viral_sequences/sequences/fasta/20220710-1633.fasta.
java -jar cdp-file-downloader.jar --domain=VIRAL_SEQUENCES --datatype=SEQUENCES --format=FASTA --location=../data/tmp/ --email=NONE --accessions=MW674860,OA971647,MT459910,MZ142846,FR995536,MZ372276,OA980617,LR882380,OA971003,OM474664,OU802805,MZ454806,OU073851,MW642481,MT511075,OA966158,OU413279,MZ278314,OA971629,OU189615,OA980908,FR996185,MW454618,OE997797,OA976746,MW883264,OA972955,LR881584,OU367928,MZ722486,MW708585,OL373368,ON326644,OA964622,OA967346,OU280039,FR995527,MW781083,OM485206,OU384682,MZ448985,OE998534,OE998028,OA980780,MW466798,MT451253,FR997114,MZ906627,FR994523,OU013039,ON051995,OA981623,ON343295,OL980598,FR991881,OD918362,OA971785,OA979875,OD999923,FR994385,LR991362,FR999305,OU081210,OU078639,FR993736,OU370354,BS000701,OU083221,LR884377,OU395444,FR998460,OA971810,OM977269,OA967564,OD966077,OA979137,FR995946,OA966559,MZ619732,OA975772,MT834314,MW882452,OL579980,OU803528,OA963966,OM

Download completed to ../data/tmp//viral_sequences/sequences/fasta/20220710-1634.fasta.
java -jar cdp-file-downloader.jar --domain=VIRAL_SEQUENCES --datatype=SEQUENCES --format=FASTA --location=../data/tmp/ --email=NONE --accessions=FR999065,MT451639,FR994908,MW681010,LR860688,OU795067,OU098152,OA973189,LC547532,OA980559,MW812765,OL588217,MW093606,OA964216,OU094891,OU801559,OU083099,MW638594,FR994652,MT459903,OA973891,MW836904,OU073679,OU084141,FR995359,OA963978,OA981487,OA973763,OA967456,MW672143,OA975975,MW986908,MT451594,OU797121,OA974542,OA978304,MW902374,MZ723149,MT263402,FR995645,OA978455,OK449084,FR999872,MW411893,MW685607,LR878252,MW707114,FR995563,OU008056,OA967061,OA980711,MZ895773,FR998055,MZ906486,OA966497,OU280287,OU539555,OA970442,LR882090,OA980865,MT811400,MZ044345,FR994825,MZ332396,MZ181297,MT750440,LR862405,MZ643442,OA964970,MW639126,FR996189,OE999209,MT371016,MZ077478,FR995281,OV005992,OA967635,OD997191,OA976567,FR995845,MZ332257,FR994462,OU084380,OA969359,MZ233161,MW

Download completed to ../data/tmp//viral_sequences/sequences/fasta/20220710-1635.fasta.
java -jar cdp-file-downloader.jar --domain=VIRAL_SEQUENCES --datatype=SEQUENCES --format=FASTA --location=../data/tmp/ --email=NONE --accessions=OU072911,MW705557,LR877769,FR999492,LC567852,OA980496,MZ231616,MZ914234,OU075669,FR996325,OU089865,MZ895772,FR999984,ON344782,OB987116,LC565413,OA976067,FR996966,FR998999,OK427141,OU368306,OU162975,FR999875,MW040651,FR993828,OA965900,OA976454,OU087993,OU075841,OU007731,OL588214,LR794424,MT612112,OU280301,FR994755,FR999900,OU809936,MW571117,OK435439,OU277853,MT882313,OU608083,OA963973,MW773877,OU181980,LR898519,FR998998,OL588215,LR824373,OU075261,OA964693,OM739270,OU077949,OU110991,OK433553,FR990238,MZ035972,OA970362,OK488049,ON071556,OA980108,OM298253,OA976434,MW539750,MZ260619,MT451441,MW286701,OA968788,OU086152,OA965486,MT706138,MZ306542,OD986591,OM739426,OA970361,ON344830,MW739088,OA964804,LR963205,FR994294,OA990138,OA965879,OA975325,OU088274,FR994852,MZ

Download completed to ../data/tmp//viral_sequences/sequences/fasta/20220710-1636.fasta.
java -jar cdp-file-downloader.jar --domain=VIRAL_SEQUENCES --datatype=SEQUENCES --format=FASTA --location=../data/tmp/ --email=NONE --accessions=OM485187,MT641760,MT506549,OA984316,OL313486,OL454579,OK401469,MW513627,MW555878,MW286725,OK234099,OK598730,OM725456,MW133976,MT520394,OK659319,MZ907919,OA965782,MW693863,MT370873,MT706174,MW935787,MW885880,MW984791,MT326164,LR881609,OK213533,MZ799056,MW172739,FR995554,OD998781,OK559112,MW053936,FR999669,ON339794,OL518870,MZ906116,MT873176,MW893875,MZ906064,MW052669,MW217410,MW425020,MT811212,MW134452,MW191150,MT653072,MZ619455,OU076633,MW751373,OK275066,OK598556,LR881540,OM739030,MW460668,FR999215,MW689904,MW053800,OK598900,MW065306,MZ433528,MZ142630,MW539730,ON166582,OK557323,MT703975,OL381478,OU576025,MW800830,OK399239,OK546741,OL852500,MZ907149,MW751249,OL313591,OA965039,MW751324,ON174277,MW540041,MZ332028,MZ905973,OM486253,OK653323,MT635443,OK510448,MW

Download completed to ../data/tmp//viral_sequences/sequences/fasta/20220710-1636.fasta.
java -jar cdp-file-downloader.jar --domain=VIRAL_SEQUENCES --datatype=SEQUENCES --format=FASTA --location=../data/tmp/ --email=NONE --accessions=MW191113,MT704070,OL408679,MW859842,MZ269101,MT811007,OA997041,MZ854426,MW493784,MW566550,MT412318,MW595916,OL756500,MW985419,MT506668,FR999855,OA963934,OL314089,OK425596,MZ229932,MW565701,OU218402,MW065288,OD997494,OW108618,OK435766,OL454591,MT811695,MW030200,OL816953,MZ266417,OU007025,MW323025,MW704393,MW751314,MT886352,MT506682,OK659687,OL313437,LR884049,MZ038143,LR824273,OL757087,MT511699,OK401784,OM211542,OK436294,OL313889,MT846464,OK598670,OL575646,MZ331086,ON166863,MZ140416,MW064428,OL756906,MT810520,OA964681,OL381119,OL313621,OL313734,OU802069,MT886383,MT412264,OL905196,MT536179,MZ917830,OL313573,OL756693,OL817240,OK435961,MZ505903,MZ141905,MT252752,OU327432,MT370945,LR884302,MW852073,MZ505911,MZ905866,MZ910219,MT811754,MW411890,MT834239,OL518869,OL